# 2D Stardist segmentation on 2D/3D/timelapse OMERO images

This notebook is used for Stardist segmentation. Some inspiration from the https://github.com/ome/omero-guide-cellprofiler/idr0002.ipynb

### Import Packages

In [64]:
# Import OMERO Python BlitzGateway
import omero
from omero.gateway import BlitzGateway
import ezomero
# Import Numpy
import numpy as np

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

#stardist related
from stardist.models import StarDist2D
from csbdeep.utils import normalize
from stardist.plot import render_label
import matplotlib.pyplot as plt
from tifffile import imsave

#load stardist model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


### Set Temp Output Directory

In [65]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())

### Setup connection with OMERO

In [66]:
conn = BlitzGateway(host='localhost', username='root', passwd='omero', secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

True


### Get info from the dataset

In [67]:
datatype = "plate" # "plate", "dataset", "image"
data_id = 52
nucl_channel = 0

#validate that data_id matches datatype
if datatype == "plate":
    plate = conn.getObject("Plate", data_id)
    print('Plate Name: ', plate.getName())
elif datatype == "dataset":
    dataset = conn.getObject("Dataset", data_id)
    print('Dataset Name: ', dataset.getName())
elif datatype == "image":
    image = conn.getObject("Image", data_id)
    print('Image Name: ', image.getName())

Plate Name:  day7


### Run Stardist on the dataset

#### Function definitions

In [ ]:
def segment_nuclei_from_slice(image):
    #normalize image
    img = normalize(image)
    labels, polygons = model.predict_instances(img)
    return labels, polygons

def save_segmentation_to_omero_as_new_image(image,labels,size_z,desc,conn): 
    new_img_name = image.getName() + "_segmentation"       
    new_img = conn.createImageFromNumpySeq(
        iter(labels), new_img_name, size_z, 1, 1, description=desc, dataset=image.getParent())
    print('Created new Image:%s Name:"%s"' % (new_img.getId(), new_img.getName()))
    #save a tif of the segmentation in new_output_directory    

def save_segmentation_to_omero_as_attach(image,labels, tmp_dir,desc,conn):
    #save segmentation to OMERO as attachment
    img_name = image.getName()
    tif_file = os.path.join(tmp_dir, img_name + "_segmentation"  + ".tif")
    imsave(tif_file, labels)
    print('Saved segmentation to:', tif_file)
    # Define the file path and namespace
    namespace = "stardist.segmentation"
    # Attach the file to the original image
    file_annotation_id = ezomero.post_file_annotation(conn, tif_file, ns=namespace, object_type="Image", object_id=image.getId(), description=desc)
    print('File annotation ID:', file_annotation_id)
    #remove tif file as it was temporary
    os.remove(tif_file)

def save_segmentation_to_omero_as_roi(image, polygons, size_z, conn):
    all_polygons = []
    
    # Handle polygons as tuple
    for z, polygon_dict in enumerate(polygons):
        if not polygon_dict or 'coord' not in polygon_dict:
            continue
            
        coords_array = polygon_dict['coord']
        if coords_array.size == 0:
            continue
            
        # Process each contour in the coordinates array
        for contour_idx in range(coords_array.shape[0]):
            try:
                # Extract x,y coordinates for current contour
                xy_coords = coords_array[contour_idx]
                #x and y coordinates are flipped from StarDist output
                points = [(float(y), float(x)) for x, y in zip(xy_coords[0], xy_coords[1])]
                
                ezomero_polygon = ezomero.rois.Polygon(
                    points=points,
                    z=z,
                    c=None,
                    t=None,
                    label="nuclei",
                    fill_color=None,
                    stroke_color=None,
                    stroke_width=None
                )
                all_polygons.append(ezomero_polygon)
            
            except Exception as e:
                print(f"Error processing contour {contour_idx} at z={z}: {e}")
                continue
    
    if all_polygons:
        try:
            roi_id = ezomero.post_roi(
                conn=conn,
                image_id=image.getId(),
                shapes=all_polygons,
                name="Stardist Nuclei",
                description="Nuclei segmentation using Stardist"
            )
            print(f"Created ROI with ID: {roi_id}")
        except Exception as e:
            print(f"Error creating ROI: {e}")
    else:
        print("Warning: No valid polygons were created")

In [140]:
import pyclesperanto_prototype as cle
import pandas as pd

def measure_intensity_per_slice(pixels, labels, size_z, size_c):
    all_statistics = []
    if size_z > 1:
        for z, label in zip(range(size_z), labels):
            for c in range(size_c):
                statistics = cle.statistics_of_labelled_pixels(pixels.getPlane(z, c, 0), label)
                statistics = pd.DataFrame(statistics)
                statistics['z'] = z
                statistics['channel'] = c
                all_statistics.append(statistics)
    else:
        statistics = cle.statistics_of_labelled_pixels(pixels.getPlane(1, 0, 0), labels)
        statistics['z'] = 0
        all_statistics.append(statistics)
    
    # Concatenate all statistics into a single DataFrame
    all_statistics_df = pd.concat(all_statistics, ignore_index=True)
    
    return all_statistics_df


#### code

In [ ]:
#TODO extend to handle multiple channels,timepoints
#TODO measure the intensity of the segmented nuclei in the other channels
#TODO attach those results as a table to the original image in OMERO

if datatype == "plate":
    wells = list(plate.listChildren())
    # use the first 3 wells only
    #wells = wells[0:3] # for testing
    well_count = len(wells)
    for count, well in enumerate(wells):
            print('Well: %s/%s' % (count + 1, well_count), 'row:', well.row, 'column:', well.column)
            # Load a single Image per Well TODO load all images for a well if there are multiple
            image = well.getImage(0)
            pixels = image.getPrimaryPixels()
            #temp
            size_c = image.getSizeC()
            size_z = image.getSizeZ()
            #get all planes for the first channel
            if size_z > 1: # 3D Image
                planes = [pixels.getPlane(z, 0, 0) for z in range(size_z)]
                labels_polygons = [segment_nuclei_from_slice(plane) for plane in planes]
                labels, polygons = zip(*labels_polygons)
                #polygons = [sublist for sublist in polygons]
            else: # 2D Image            
                print("2D")   
            #save stack back to OMERO same project only add _nucleisegmentation to the name
            new_img_name = image.getName() + "_nucleisegmentation"
            desc = "Stardist nuclei segmentation"
            all_statistics_df = measure_intensity_per_slice(pixels,labels,size_z,size_c)
            tabelid = ezomero.post_table(conn, object_type="Image", object_id=image.getId(), table = all_statistics_df,title="Nuclei_measurements")
            print('Created table ID:', tabelid)
            #save_segmentation_to_omero_as_attach(image,labels, new_output_directory,desc,conn)
            save_segmentation_to_omero_as_new_image(image,labels,size_z,desc,conn)
            save_segmentation_to_omero_as_roi(image,polygons,size_z,conn)
            
elif datatype == "dataset":
    images = list(dataset.listChildren())
    # use the first 3 images only
    images = images[0:3]
    image_count = len(images)
    for count, image in enumerate(images):
        print('Image: %s/%s' % (count + 1, image_count), 'Name:', image.getName())
        pixels = image.getPrimaryPixels()
        size_c = image.getSizeC()
        nuc_img = pixels.getPlane(0, 0, 0)

Well: 1/72 row: 0 column: 0


cast.py (1134): invalid value encountered in cast
cast.py (1153): invalid value encountered in cast


Created table ID: 263
Created new Image:726 Name:"14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 01)].tif_segmentation"
Created ROI with ID: 16
Well: 2/72 row: 0 column: 1


cast.py (1134): invalid value encountered in cast
cast.py (1153): invalid value encountered in cast


Created table ID: 264
Created new Image:727 Name:"14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 02)].tif_segmentation"
Created ROI with ID: 17
Well: 3/72 row: 0 column: 2


...

Created table ID: 291
Created new Image:754 Name:"14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 29)].tif_segmentation"
Created ROI with ID: 44
Well: 30/72 row: 2 column: 5
Created table ID: 292
Created new Image:755 Name:"14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 30)].tif_segmentation"
Created ROI with ID: 45
Well: 31/72 row: 2 column: 6


cast.py (1134): invalid value encountered in cast
cast.py (1153): invalid value encountered in cast


Created table ID: 293
Created new Image:756 Name:"14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 31)].tif_segmentation"
Created ROI with ID: 46
Well: 32/72 row: 2 column: 7


cast.py (1134): invalid value encountered in cast
cast.py (1153): invalid value encountered in cast


Created table ID: 294
Created new Image:757 Name:"14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 [14-6-24_FUCCI_PDLO_Day7_plate1-fixed_007.nd2 (series 32)].tif_segmentation"
Created ROI with ID: 47
Well: 33/72 row: 2 column: 8
